# MCFR-D
*Data taken from the mcfr.xslx spreadsheet in the repo (originally from TerraPower Report)*

Note: Comment/Uncomment a block by typing ```ctrl + /```

In [1]:
import openmc
import numpy as np
import matplotlib.pyplot as plt

## Materials

In [2]:
## fuel material specifications
U_atom_percent  = 0.25*(1/3)
U_enrichment    = 19.75 # %U235
Cl_atom_percent = 0.75*(1/3) + 0.5*(2/3)
Cl_enrichment   = 99.0 # %Cl37
Na_atom_percent = 0.5*(2/3)

fuel_T          = 695 + 273.15 # K
A               = 4212.6
B               = 1.0686
fuel_density    = (A - B*fuel_T ) * (1000/1) * ((1/100)**3) # g / cc

## clad material specifications
clad_elements = ["Ni", "Cr", "Mo", "Fe", "Nb", "Ta", "Co", "Mn", "Si", "Al", "Ti", "C", "P", "S"]
clad_element_weight_percents = [0.58, 0.2142, 0.09, 0.05, 0.01825, 0.01825, 0.01, 0.005, 0.005, 0.004, 0.004, 0.001, 0.00015, 0.00015]
clad_T = 695 + 273.15 # K
clad_density = 8.3 # g / cc

## reflector material specifications
reflector_T = 695 + 273.15 # K
reflector_density = 3.6 # g / cc
reflector_atom_percent_Mg = 0.5
reflector_atom_percent_O = 0.5

In [3]:
fuel = openmc.Material(name="fuel")
fuel.add_element('U', U_atom_percent, percent_type='ao', enrichment=U_enrichment)
fuel.add_element('Cl', Cl_atom_percent, percent_type='ao', enrichment=Cl_enrichment, enrichment_target='Cl37', enrichment_type='wo')
fuel.add_element('Na', Na_atom_percent, percent_type='ao')
fuel.set_density('g/cm3', fuel_density) # set density
fuel.temperature = fuel_T

clad = openmc.Material(name="clad")
for i in range(len(clad_elements)):
    clad.add_element(clad_elements[i], clad_element_weight_percents[i], percent_type='wo')
clad.set_density('g/cm3', clad_density)
clad.temperature = clad_T

reflector = openmc.Material(name="reflector")
reflector.add_element('Mg', reflector_atom_percent_Mg, percent_type='ao')
reflector.add_element('O',  reflector_atom_percent_O,  percent_type='ao')
reflector.set_density('g/cm3', reflector_density)
reflector.temperature = reflector_T

/home/william/miniconda3/envs/openmc_env/lib/python3.13/site-packages/openmc/material.py:805: UserWarning: A uranium enrichment of 19.75 was given for Material ID="1". OpenMC assumes the U234/U235 mass ratio is constant at 0.008, which is only valid at low enrichments. Consider setting the isotopic composition manually for enrichments over 5%.
  warnings.warn(msg)


In [4]:
materials = openmc.Materials([fuel, clad, reflector])
materials.export_to_xml()

## Surfaces

In [5]:
cylinder_radii = [95.0, 96.0, 185.0]
zplane_offsets = [-265.0, -176.0, -175.0, 175.0, 176.0, 265.0]

In [6]:
c1  = openmc.ZCylinder(r=cylinder_radii[0])
c2  = openmc.ZCylinder(r=cylinder_radii[1])
c3  = openmc.ZCylinder(r=cylinder_radii[2], boundary_type='vacuum')
z_3 = openmc.ZPlane(z0=  zplane_offsets[0], boundary_type='vacuum')
z_2 = openmc.ZPlane(z0=  zplane_offsets[1])
z_1 = openmc.ZPlane(z0=  zplane_offsets[2])
z1  = openmc.ZPlane(z0=  zplane_offsets[3])
z2  = openmc.ZPlane(z0=  zplane_offsets[4])
z3  = openmc.ZPlane(z0=  zplane_offsets[5], boundary_type='vacuum')

## Cells

In [7]:
bottom_reflector_region = +z_3 & -z_2 & -c3
top_reflector_region    = +z2 & -z3 & -c3
radial_reflector_region = +z_2 & -z2 & +c2 & -c3
bottom_clad_region      = +z_2 & -z_1 & -c2
top_clad_region         = +z1 & -z2 & -c2
axial_clad_region       = +z_1 & -z1 & +c1 & -c2
total_core_region       = +z_1 & -z1 & -c1

In [8]:
bottom_reflector = openmc.Cell(fill = reflector, region = bottom_reflector_region)
top_reflector    = openmc.Cell(fill = reflector, region = top_reflector_region)
radial_reflector = openmc.Cell(fill = reflector, region = radial_reflector_region)
bottom_clad      = openmc.Cell(fill = clad,      region = bottom_clad_region)
top_clad         = openmc.Cell(fill = clad,      region = top_clad_region)
axial_clad       = openmc.Cell(fill = clad,      region = axial_clad_region)
total_core       = openmc.Cell(fill = fuel,      region = total_core_region)

In [9]:
root_universe = openmc.Universe(cells=[bottom_reflector, top_reflector, radial_reflector, bottom_clad, top_clad, axial_clad, total_core])
geometry = openmc.Geometry()
geometry.root_universe = root_universe
geometry.export_to_xml()

## Visualization
[OpenMC Documentation on Visualization](https://docs.openmc.org/en/stable/usersguide/plots.html)

In [10]:
## Inline plots
# root_universe.plot(width=(400, 400), basis='xy')
# root_universe.plot(width=(400, 550), basis='xz')

In [11]:
# xy basis png plot
xy_png          = openmc.Plot(name="xy_png")
xy_png.basis    = 'xy'
xy_png.origin   = (0, 0, 0)
xy_png.width    = (400, 400)
xy_png.pixels   = (700, 700)
xy_png.color_by = 'material'
xy_png.filename = 'xy_png'

# xz basis png plot
xz_png          = openmc.Plot(name="xz_png")
xz_png.basis    = 'xz'
xz_png.origin   = (0, 0, 0)
xz_png.width    = (400, 550)
xz_png.pixels   = (700, 700)
xz_png.color_by = 'material'
xz_png.filename = 'xz_png'

# voxel plot
voxel_plot          = openmc.Plot(name="voxel_plot")
voxel_plot.type     = 'voxel'
voxel_plot.origin   = (0,0,0)
voxel_plot.width    = (400, 400, 550)
voxel_plot.pixels   = (400, 400, 550)
voxel_plot.filename = 'voxel_plot'

In [12]:
visualization = openmc.Plots([xy_png, xz_png, voxel_plot])
visualization.export_to_xml()

## Settings

In [13]:
point = openmc.stats.Point((0, 0, 0))
source = openmc.IndependentSource(space=point)

settings = openmc.Settings()
settings.run_mode = 'eigenvalue'
settings.source = source
settings.batches = 150
settings.inactive = 20
settings.particles = 1000

settings.temperature = {'method': 'interpolation'}

In [14]:
settings.export_to_xml()

## Tallies
Create energy bins to seperate tallies into (ie. count a neutron in bin n if it has energy between $E_n$ and $E_{n+1}$).

Create Tallies to calculate:
- Flux
- Total Reaction Rate
- Fission Rate
- Absorption Rate

In [15]:
## create the bins for the energy filter as a list in python to input into the OpenMC energy filter object
anl33_bins = [
              1.019348e-05, 0.4184023, 0.5322266, 3.929497, 8.315325, 13.71028, 22.60361,
              37.2681, 61.44724, 101.3026, 167.0288, 275.3745, 454.0105, 748.5285, 1234.154,
              2034.725, 3354.714, 5530.878, 9118.95, 15035.79, 24791.03, 40869.3, 67383.99,
              111160, 183183.5, 301983.3, 497960, 821165.3, 1354100, 2231400, 3679502, 6066000,
              10000000, 14227500
              ]

In [16]:
material_filter = openmc.MaterialFilter([fuel]) # filter to only count neutrons in the fuel material
energy_filter = openmc.EnergyFilter(anl33_bins) # energy filter based on bins we created

In [17]:
## Note: we name the tallies so that later in post-processing we can more easily access them

## Tally Flux
flux_tally = openmc.Tally(name="flux")
flux_tally.filters = [material_filter, energy_filter]
flux_tally.scores = ['flux']

## Tally Total Reaction Rate
total_tally = openmc.Tally(name="total")
total_tally.filters = [material_filter, energy_filter]
total_tally.scores = ['total']

## Tally Fission Rate
fission_tally = openmc.Tally(name="fission")
fission_tally.filters = [material_filter, energy_filter]
fission_tally.scores = ['fission']

## Tally Absorption Rate
absorption_tally = openmc.Tally(name="absorption")
absorption_tally.filters = [material_filter, energy_filter]
absorption_tally.scores = ['absorption']

In [18]:
## create tallies.xml
tallies = openmc.Tallies([flux_tally, total_tally, fission_tally, absorption_tally])
tallies.export_to_xml()

## Run File - No Depletion

In [19]:
## Generate Plots
# openmc.plot_geometry()

In [20]:
## Convert Voxel plot to vtk
## (make sure you have the python vtk package and install praview onto your computer)
# openmc.voxel_to_vtk(voxel_file = f'{voxel_plot.filename}.h5', output = voxel_plot.filename)

In [21]:
## Run data (uncomment the following line)
# openmc.run()

## Post Processing - Tallies

OpenMC has great built-in tools for post processing. It will read the tallies output from the .h5 file which makes it easier for us to interact with the data.

Note: When running OpenMC with regulare python files (not a notebook like jupyter), the post processing would be done in a seperate python file from the model itself.

Also Note: The units of the tallies are normalized per source particle, so in many cases it takes a lot of effort to get physically meaningful results, but our tally results can still be extremely useful without full physical scaling.

- There will be a 'tallies.out' file with a nice visual representation of the results of your tallies
- The more important output will be found in your statepoint file (the number in the statepoint is the number of batches)
- (summary.h5 is a summary of your input)

In [22]:
! ls

geometry.xml   mcfr_D.ipynb  settings.xml	summary.h5   tallies.xml
materials.xml  plots.xml     statepoint.150.h5	tallies.out


In [23]:
## Use the build in OpenMC statepoint object and pass into it the name of your statepoint
statepoint = openmc.StatePoint('statepoint.150.h5')

## Have python read the tally output data from the statepoint file
my_flux       = statepoint.get_tally(name="flux")
my_total      = statepoint.get_tally(name="total")
my_fission    = statepoint.get_tally(name="fission")
my_absorption = statepoint.get_tally(name="absorption")

In [24]:
## You can view data about the tally by using print(tally_name)
# print(my_flux)

In [25]:
## my_tally.mean will display the mean value of the tally
# print(my_flux.mean)
# print(my_flux.mean.shape)

## my_tally.std_dev will display the standard deviation of the tally
# print(my_flux.std_dev)
# print(my_flux.std_dev.shape)

#### Here, we'll save the values of the flux tally to a list and then plot it in python

In [26]:
## since we defined the energy filter in the input of the model, we now access it by reading the output
energy_filter = my_flux.find_filter(openmc.EnergyFilter)
energy_bins = energy_filter.bins  # This is a list of (E_min, E_max) tuples

## we will plot the tallies based on the midpoints of the energy bins
## (we could do this as a histogram because we seperated the tallies into discrete bins,
## but we'll just linearly interpolate between each point and use plt.plot instead).
energy_bin_midpoints = [(low + high)/2 for low, high in energy_bins]

In [27]:
# Mean values of the tally (shape depends on filters)
flux_means = my_flux.mean.ravel()  # flatten if necessary
flux_stds = my_flux.std_dev.ravel()

In [28]:
## now create a plot of the flux as a function of neutron energy using our tally data (uncomment the following lines)
# plt.errorbar(energy_bin_midpoints, flux_means, yerr=flux_stds, fmt='-o', capsize=4, markersize=4)
# plt.xscale('log')
# plt.yscale('linear')
# plt.xlabel("Energy (eV)")
# plt.ylabel("Flux")
# plt.title("Fuel Neutron Flux vs Energy")
# plt.grid(True, which='both', ls='--')
# plt.show()

In [29]:
## We can now save the figure as a png file (uncomment the following line)
# plt.savefig("mcfr_D_flux.png", dpi=800)

#### We'll now plot the other three tallies

In [30]:
## Plot Total Reaction Rate
# total_means = my_total.mean.ravel()
# total_stds = my_total.std_dev.ravel()
# plt.errorbar(energy_bin_midpoints, total_means, yerr=total_stds, fmt='-o', capsize=4, markersize=4)
# plt.xscale('log')
# plt.yscale('linear')
# plt.xlabel("Energy (eV)")
# plt.ylabel("Total Reaction Rate")
# plt.title("Fuel Total Reaction Rate vs Energy")
# plt.grid(True, which='both', ls='--')
# plt.show()
# plt.savefig("mcfr_D_total.png", dpi=800)

## Plot Fission Rate
# fission_means = my_fission.mean.ravel()
# fission_stds = my_fission.std_dev.ravel()
# plt.errorbar(energy_bin_midpoints, fission_means, yerr=fission_stds, fmt='-o', capsize=4, markersize=4)
# plt.xscale('log')
# plt.yscale('linear')
# plt.xlabel("Energy (eV)")
# plt.ylabel("Fission Rate")
# plt.title("Fuel Fission Rate vs Energy")
# plt.grid(True, which='both', ls='--')
# plt.show()
# plt.savefig("mcfr_D_fission.png", dpi=800)

## Plot Absorption Rate
# absorption_means = my_absorption.mean.ravel()
# absorption_stds = my_absorption.std_dev.ravel()
# plt.errorbar(energy_bin_midpoints, absorption_means, yerr=absorption_stds, fmt='-o', capsize=4, markersize=4)
# plt.xscale('log')
# plt.yscale('linear')
# plt.xlabel("Energy (eV)")
# plt.ylabel("Absorption Rate")
# plt.title("Fuel Absorption Rate vs Energy")
# plt.grid(True, which='both', ls='--')
# plt.show()
# plt.savefig("mcfr_D_absorption.png", dpi=800)

In [31]:
# ! ls

## Run Depletion

Note: The 'integrator' is pretty much which numerical method should be used to solve the depletion calculation. That's outside of our scope for now. Just know that probably the best operator is CECM, but the simplest/fastest is probably Predictor Integrator.

Note: The 'operator' is what connects transport calculations to depletion calculations. It links the geometry, settings, and chain files.

When you run a depletion you need:
- A specified power that you're running your reactor at
- Specified timestep units (standard is MW Days / kg Heavy Metal)
- Specified timesteps to do depletion calculations at
- The volume must be specified for any depletable materials

Also, for the depletion to run, the xml files must be created. In this case, we already created them, so we can go ahead and run the depletion calculation.

In [32]:
import openmc.deplete

In [33]:
## Set the paramaters of our depletion calculation
power = 150000.0 # Watts
timestep_units='MWd/kg'
steps=[0.1, 0.5, 1.0, 5.0, 10.0, 15.0, 100.0] # Note: no need to specify the 0 depletion step
fuel_volume = np.pi * 95**2 * 350 # cm^3 (volume of fuel cylinder)

In [34]:
## We want all our .xml files
## and we want our objects that we created for materials, geometry, and settings

## NOTE also that we now need to specify the volume of the depletable material (in this case just the fuel)
fuel.volume = fuel_volume
materials = openmc.Materials([fuel, clad, reflector])
materials.export_to_xml()

root_universe = openmc.Universe(cells=[bottom_reflector, top_reflector, radial_reflector, bottom_clad, top_clad, axial_clad, total_core])
geometry = openmc.Geometry()
geometry.root_universe = root_universe
geometry.export_to_xml()

settings.export_to_xml()

In [35]:
## You need to create an openmc.Model object that includes the geometry and settings of our model
## The 'geometry' and 'settings' objects are the same ones that we created earlier to and used to make the xml files
MCFR_D_model = openmc.Model(geometry=geometry, settings=settings)

## you then pass the model into your operator using the CoupledOperator object
operator = openmc.deplete.CoupledOperator(MCFR_D_model)

In [36]:
## You then create in OpenMC integrator object with specified paramaters
integrator = openmc.deplete.CECMIntegrator(operator=operator,
                                           timesteps=steps,
                                           power=power,
                                           timestep_units=timestep_units)

In [37]:
## Now run the depletion
## (You may not actually be able to run the depletion on your personal machine due to limited computing power)
# (uncomment the following line)
# integrator.integrate()

## Post Processing - Depletion

You can do many things in post processing with depletion data. A couple common/simple things are:
- You can plot keff as a function of burnup/time
- You can plot the concentration of different isotopes as a function of burnup/time

In [38]:
## Might not work if you weren't able to run a depletion on your computer

## Access results using the OpenMC results object
# MCFR_D_results = openmc.deplete.Results("depletion_results.h5")

In [39]:
## Extract the keff value at each burnup step
# time, k_and_stdev = MCFR_D_results.get_keff(time_units='d') # keff values coupled with corresponding timesteps

# keff  = k_and_stdev[:, 0]
# stdev = k_and_stdev[:, 1]

# plt.errorbar(steps, keff, yerr=stdev_openmc, fmt='o', capsize=5)

In [40]:
## Extract the concentration of U235 at each burnup step
# _, concentrations_over_time = results_openmc.get_atoms("1", "U235")

# plt.plot(burnup_steps_openmc, concentrations_over_time, label='Concentration of U235 vs Burnup')